Aula 05: Utilização do Python + OpenCV no futebol de robôs

Nessa aula iremos aprender como implementar os conhecimentos transmitidos no curso, para o vsss (Very Small size Soccer). Primeiro passo é compreender como funcionam os elementos do futebol de robôs.

O _Very Small Size soccer (VSSS)_ tem como objetivo a competição de "futebol" entre times de robôs autônomos. Cada time é constituído de três robôs de até 7,5x7,5x7,5 cm de dimensão. O objetivo de cada time é fazer com que a bola entre no gol do time adversário de forma autônoma, através do desenvolvimento de tecnologias de inteligência artificial, reconhecimento de imagem, eletrônica e mecânica. O foco da competição da VSSS está na cooperação e no controle multi-agente inteligente em um ambiente altamente dinâmico com um sistema híbrido (centralizado/distribuído) e no reconhecimento dos robôs pela visão. Ou seja, é necessário fazer com que os robôs joguem coletivamente ao realizar passes e jogadas.

<img src="assets/foto_vss.png" width="500"/>

A visão é formada por uma câmera para cada time, cada um com o seu próprio sistema de visão para rastrear todos os objetos no campo. Cada equipe possui o seu computador que recebe os comandos do juiz e deve inferir a posição dos robôs nos dados enviados pela câmera. Os computadores fazem o processamento necessário para a coordenação e controle dos robôs sendo a comunicação com os robôs feita via wireless.

In [1]:
import numpy as np
import cv2

In [2]:
def camera_test(camera):
    if camera:
        # if iyou want to use camera 
        cap = cv2.VideoCapture(0)
        print(cap)
    else:
        # use video file instead of camera
        cap = cv2.VideoCapture('img/sample_ufpbots.mp4')
    return cap

In [3]:
cap = camera_test(True)

<VideoCapture 0x7fb52e7fa430>


In [4]:
cap.release()

## 5.1 - Separando um objeto no campo

Para separarmos um objeto no campo utilizando o openCV, faremos uso de algumas funções para identificar diferentes areas em um vídeo (ou frames oriundos de uma câmera). Essas funções são têm diferentes classificações na estrutura da biblioteca e executam tarefas diferentes cada uma delas. As funções que serão utilizadas nesse exemplo são:

* `cv2.cvtColor(variavelimagem, conversão)`
* `cv2.inRange(variavelimagem, pontomin, pontomax)`
* `cv2.contourArea(contorno)`
* `cv2.findContours(mascara, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)`
* `cv2.erode(mask, None, iterations=2)`
* `cv2.dilate(mask, None, iterations=2)`


### 5.1.1 - cvtColor

parâmetros : 
* **variavelimagem** é onde colocamos a imagem, vídeo ou captura de imagem;
* **conversão** é onde colocamos o parâmetro que vai transformar de BGR para HSV(hue (matiz), saturation (saturação) e value (valor).), no nosso caso iremos usar o cv2.COLOR_BGR2HSV

Agora, com a função inRange iremos criar uma máscara que vai, filtrar e mostrar apenas os valores de cores delimitados por um intervalo que vamos definir.


### 5.1.2 - inRange

Antes de mostrarmos os parâmetros dessa função, iremos ter que definir dois vetores multidimensionais utilizando a função np.array([hue,saturation,value]),criaremos um vetor para a intensidade máxima do objeto( campo) outro pra a intensidade mínima. Tendo isso em mente podemos mostrar os pârametros do inRange.

Parâmetros : 
* **variavelimagem** :é onde colocamos a imagem, vídeo ou captura de imagem;
* **pontomin** : é aquele vetor que contém a intensidade mínima da cor que estamos procurando;
* **pontomax** : é o vetor que contém a intensidade máxima da cor que estamos procurando.
             
Dito isso, para acharmos a borda do campo, temos que ver qual o intervalo de HSV se encontra esse branco, que podemos verificar na janela do opencv. Para encontrar a bolinha é da mesma forma, só que mudaremos o intervalo do HSV.

### 5.1.3 - contourArea

A partir dos contornos achados iremos delimitar a área de cada possível contorno, aplicando a funçâo cv2.contourArea para cada contorno achado, fomando junto com outros comandos um vetor que armazena os contornos. A linha de código fica:

### 5.1.4 - findContours

Ja sabemos como encontrar a bolinha, porém agora precisamos marca-la de alguma forma no campo, no nosso caso iremos usar um retângulo, mas para marcarmos a bolinha temos que determinar seus contornos com a função cv2.findContours.

Parâmetros: 
* **mascara**: máscara criada com o inRange;
* **cv2.RETR_TREE**: constrói uma hierarquia entre os contornos e concatena-os como uma pirâmide;
* **cv2.CHAIN_APPROX_SIMPLE**: essa função comprime os segmentos verticais,horizontais e diagonais , deixando apenas seus pontos finais, como por exemplo num retângulo é deixado 4 pontos.

### 5.1.5 - erode

### 5.1.6 - dilate

## 5.2 - Exemplo

In [5]:
cap = camera_test(False)

In [6]:
while(cap.isOpened()):
    
    rfm, frame = cap.read()
    if not rfm:
        break
    HSV = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    BRANCO = np.array([150,50,255])
    branco = np.array([84,2,174])
    mascara = cv2.inRange(HSV,branco,BRANCO)
    
    maior_laranja = np.array([40,170,255])
    menor_laranja = np.array([20,120,255])
    mascara_bola = cv2.inRange(HSV,menor_laranja,maior_laranja)
    
    campo = mascara[62:434,79:558]
    
    try:
        contours, hierarchy = cv2.findContours(mascara_bola, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
        biggest_contour = max(contour_sizes, key=lambda x: x[0])[1]
        x, y, w, h = cv2.boundingRect(biggest_contour)
        frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
    except:
        print("a bola esta coberta")
    
    
    cv2.imshow("campo",campo)
    cv2.imshow("bola", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
a bola esta coberta
